# Exercise 3.3

Our frog has found itself in a new and strange environment: its own home! Even this simple environment poses a challenge for our frog's existing behaviour. All of the flies are in the pantry, and it's locked, and furthermore, the frog doesn't even know how to pick up a key. Let's help it!

### First, Install the Lab Libraries

In [ ]:
%pip install python_actr git+https://github.com/eilene-ftf/pondworld.git

  Cloning https://github.com/eilene-ftf/pondworld.git to /tmp/pip-req-build-d2zsgd1t
  Running command git clone --filter=blob:none --quiet https://github.com/eilene-ftf/pondworld.git /tmp/pip-req-build-d2zsgd1t
  Resolved https://github.com/eilene-ftf/pondworld.git to commit 7aa712ff4d7ad0034d2e95090831542552845893
  Preparing metadata (setup.py) ... done


### Second, Import the Necessary Libraries

In [ ]:
import gymnasium as gym
from minigrid.minigrid_env import MiniGridEnv

import pondworld
from pondworld import FrogControl, EnvState

from python_actr import Model, ACTR, Buffer, Memory, log_everything

from IPython.display import clear_output

import time

### The Pond

**Don't forget to run this bit!**

In [ ]:
class Pond(Model):
    pass

### The Frog

Now that you're comfortable with the mental world of the frog, we're going to try to tell the frog what to do about getting into that pantry. The door is locked, and our frog's left a key somewhere around the house. As it cannot open the door without a key, the first thing to do is get the key. Try running the code first to see what's happening in the frog's world, and then go look at your task.

In [ ]:
# create our frog's brain
class FrogMind(ACTR):
    # give our frog a buffer that holds information
    # let's call this buffer "vision". it represents what the frog is seeing.
    vision=Buffer()

    # here we're going to set what the frog is seeing right at the moment
    # initially we don't know what the frog is looking at
    vision.set('unknown')

    # "possibilities" has been replaced with a set of slot names, corresponding
    # to the three items the frog might track in its world, and a set of values,
    # corresponding to what states those items might be in
    slots = ["fly", "key", "door"]
    values = ["nowhere", "here", "ahead", "left", "right", "held"]

    # also, we're keeping track of all the things the frog can do
    # you never know when you might need them
    actions = ["forward", "left", "right", "interact", "pickup"]

    ######
    # THE BASICS

    # look to see if there's any flies
    def look(vision='unknown'):
        self.wait(0.3) # adds a rest between steps so we can see what's going on
        self.parent.clear() # makes sure only the most recent output is shown
        observation = self.parent.frog_body.look()  # now, when it looks,
                                                    # the frog will get a
                                                    # collection of several
                                                    # items

        # This creates a list of 'slot:value' pairs, using what's
        # contained in observation
        new_memory = []
        for slot in slots:
            val_index = observation[slot]   # gets a number indicating the state
                                            # of the object called 'slot'
            value = values[val_index]       # then looks it up
            new_memory.append(f'{slot}:{value}') # and adds it to

        updated = ' '.join(new_memory)  # make the contents of new_memory into a
                                        # string ACT-R can understand
        print('what the frog sees: \n  ' + updated)
        vision.set(updated)

    # if you see a fly, eat a fly
    def eat_fly(vision='fly:here'):
        print("The frog spits out its tongue and snaps up the fly.")
        result = self.parent.frog_body.move("interact")
        if result.terminated:
          vision.set('done') # the frog has satisfied its hunger
        else:
          vision.set('unknown')

    ######
    # EXPLORATION

    # if you don't see a fly try looking left
    def explore_left(vision="fly:nowhere"):
        print("The frog explores left.")
        self.parent.frog_body.move("left")
        vision.set('unknown')

    # if you don't see a fly try hopping forward
    def explore_forward(vision="fly:nowhere"):
        print("The frog explores forward.")
        self.parent.frog_body.move("forward")
        vision.set('unknown')

    ######
    # CHASE THE FLY

    # if fly is to the left, turn left
    def turn_left_fly(vision="fly:left"):
        print("The frog turns left.")
        self.parent.frog_body.move("left")
        vision.set('unknown')

    # if fly is to the right, turn right
    def turn_right_fly(vision="fly:right"):
        print("The frog turns right.")
        self.parent.frog_body.move("right")
        vision.set('unknown')

    # if you see something in the distance trying hopping forward
    def hop_forward_fly(vision="fly:ahead"):
        print("The frog hops forward.")
        self.parent.frog_body.move("forward")
        vision.set('unknown')


    ######
    # GRAB THE KEY - PUT FOUR NEW PRODUCTIONS HERE


    ######
    # END STATE

    # when all the flies are eaten, it's time to stop and rest
    def end(vision='done'):
        print("The frog is full.")
        self.stop()

### Your Task

Here, your goal is to **add new productions** to the frog, such that it behaves with the **key** a little bit like it behaves with the **flies**. When the frog sees the key, it should go straight there, and try to pick it up. There is one key difference, however: you're going to teach the frog to pick up its key. **Add this production to the frog**:

```py
def pickup_key(vision='key:here'):
    print('The frog picks up the key to the pantry.')
    self.parent.frog_body.move('pickup')
    vision.set('unknown')
```

Instead of using `'interact'`, picking up a key uses the `'pickup'` command. Pretty sensible, but you'll want to see what happens to `vision` after the frog has picked up the key. In order to (reliably) see what the frog does, **add three more productions**, based on the frog's behaviour when it sees a fly, that **make the frog go to a key when it sees one, in order to pick it up**.

There is one more thing you need to change in order to make this all work consistently. In order for the frog to dependably go to the key and pick it up, it must **not** choose to explore when it sees the key. Therefore, **the exploration productions must only fire when the frog does not see a key**.

Recalling that each production rule fires based on a *condition* that you assert when you describe it, **what should you add to the existing condition that will keep the frog from wandering**? Here is an example state vision may be in to help you figure it out:

```py
'fly:nowhere key:ahead door:left'
```

There is a print statement that has been added to the frog, it shows the current state of memory. What is different before and after the frog picks up the key?

### Run Your Model

Make sure everything works by running the code blocks below.

In [ ]:
# Sets up the frog's virtual world
env = gym.make("house-v0", tile_size=32, render_mode='rgb_array')
# Gives the frog a body in it
frog_body = FrogControl(env, textmode=True, emojis=True)
# Gets things going!
frog_body.start()

In [ ]:
kermit = FrogMind()                       # name the agent
kermit.wait = time.sleep
paradise_swamp = Pond()                   # name the environment
paradise_swamp.clear = lambda: clear_output(wait=True)
paradise_swamp.frog_body = frog_body      # put the frog's body in the environment
paradise_swamp.agent = kermit             # put the frog's brain in the environment
paradise_swamp.run()                      # Annnnd action!

🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    
🧱⬛⬛🪰⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    
🧱⬛⬛⬛⬛🪰⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🐸🧱    
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    live frog reaction:
🧱⬛🪰⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    ⬛⬛⬛⬛⬛⬛⬛
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🟨🧱    ⬛⬛⬛⬛⬛⬛⬛
🧱🪰⬛⬛⬛⬛🪰⬛🧱⬛⬛⬛⬛🗝️⬛🧱    ⬛⬛⬛⬛⬛⬛⬛
🧱⬛⬛⬛🪰⬛⬛⬛🚪⬛⬛⬛⬛⬛⬛🧱    ⬛⬛⬛⬛⬛⬛⬛
🧱🪰⬛⬛⬛⬛⬛🪰🧱⬛⬛⬛⬛⬛⬛🧱    ⬛⬛⬛⬛⬛⬛⬛
🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱    🧱🧱🧱🧱🧱🧱🧱
🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    🟨🟨🟨🐸🟨🟨🟨
eat all the flies
frog compass: ➡️
what the frog sees: 
  fly:nowhere key:nowhere door:nowhere
The frog explores forward.


KeyboardInterrupt: 